# Data preprocessing: more art than science?

## Contents of this notebook:
<ol>
<li>Load and examine your data</li>
<li>Documentation</li>
<li>Removing variables (columns) that you don't need</li>
<li>One-hot encoding or binarizing categorical variables</li>
<li>Making your data machine-readable</li>
<li>Handling not available (NA) and inf data</li>
<li>Removing columns with a standard deviation of 0</li>
<li>Feature Scaling</li>
<li>Loading local files into Google Colab</li>
</ol>

## The Data
Restricted (but open-access) data from the [Human Connectome Project](http://www.humanconnectomeproject.org/data/) S1200 Release of Young Adult Twin and Non-Twin Siblings

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

## 1. Load and examine your data

In [ ]:
df = pd.read_csv("HCP_restricted_data.csv")

/Users/nadiablostein/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (51,52,123,126,127,134,135,587) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1086 entries, 0 to 1085
Columns: 597 entries, Unnamed: 0 to Mars_Final
dtypes: bool(54), float64(406), int64(39), object(98)
memory usage: 4.6+ MB


In [ ]:
df.describe()

,Unnamed: 0,Subject,Gender,Age_in_Yrs,Mother_ID,Father_ID,TBV,Gp_Left,Gp_Right,Str_Left,...,NEOFAC_E,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final
count,1086.000000,1086.000000,1086.000000,1086.000000,1086.000000,1086.000000,1.086000e+03,1086.000000,1086.000000,1086.000000,...,1079.000000,1084.000000,1084.000000,1081.000000,1085.000000,1080.000000,1080.000000,1078.000000,1075.000000,1075.000000
mean,542.500000,372899.458564,1.456722,28.787293,53149.575506,83198.063536,1.406181e+06,1527.126096,1407.109537,10083.244761,...,30.678406,110.508515,97.856375,1.402405,45.712166,94.955509,93.784704,1.847078,0.580465,1.823916
std,313.645501,272924.716578,0.498353,3.696538,3614.307194,3117.170954,1.512145e+05,155.784980,146.468185,1045.451066,...,5.998099,9.048125,11.149398,1.722926,7.582800,14.490435,14.761768,0.570432,0.986779,0.572075
min,0.000000,100206.000000,1.000000,22.000000,50263.000000,80216.000000,1.006870e+06,1102.740000,987.497000,6868.920000,...,10.000000,82.740000,59.860000,0.000000,38.600000,56.350000,59.500000,1.560000,0.000000,1.080000
25%,271.250000,154007.750000,1.000000,26.000000,51593.000000,81464.000000,1.295598e+06,1417.705000,1306.140000,9333.285000,...,27.000000,101.120000,87.110000,0.000000,38.600000,84.070000,83.120000,1.800000,0.000000,1.760000
50%,542.500000,211770.500000,1.000000,29.000000,52302.500000,82201.000000,1.393265e+06,1519.830000,1402.180000,10017.850000,...,31.000000,108.790000,98.040000,1.000000,45.900000,95.360000,94.970000,1.800000,0.000000,1.800000
75%,813.750000,585030.750000,2.000000,32.000000,54162.000000,84459.000000,1.519470e+06,1631.990000,1501.310000,10759.450000,...,35.000000,122.250000,110.450000,2.000000,52.200000,104.660000,102.470000,1.880000,1.000000,1.840000
max,1085.000000,996782.000000,2.000000,37.000000,99998.000000,99999.000000,1.929300e+06,2043.250000,1935.210000,13572.200000,...,47.000000,122.250000,111.410000,10.000000,75.300000,134.650000,131.380000,15.000000,17.000000,15.000000


In [ ]:
df['ZygositySR'].unique()

array(['NotTwin', 'NotMZ', 'MZ', ' '], dtype=object)

In [ ]:
# All the different types of penguin species (union of column data)
df['ZygositySR'].value_counts()

NotTwin    528
NotMZ      285
MZ         269
             4
Name: ZygositySR, dtype: int64

In [ ]:
df.columns

Index(['Unnamed: 0', 'Subject', 'Gender', 'Age_in_Yrs', 'HasGT', 'ZygositySR',
       'ZygosityGT', 'Family_ID', 'Mother_ID', 'Father_ID',
       ...
       'Noise_Comp', 'Odor_Unadj', 'Odor_AgeAdj', 'PainIntens_RawScore',
       'PainInterf_Tscore', 'Taste_Unadj', 'Taste_AgeAdj', 'Mars_Log_Score',
       'Mars_Errs', 'Mars_Final'],
      dtype='object', length=597)

In [ ]:
df.head()

,Unnamed: 0,Subject,Gender,Age_in_Yrs,HasGT,ZygositySR,ZygosityGT,Family_ID,Mother_ID,Father_ID,...,Noise_Comp,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final
0,0,100206,2,27,True,NotTwin,,56037_85858,56037,85858,...,6,108.79,97.19,1.0,49.7,72.63,72.03,1.84,0.0,1.84
1,1,100307,1,27,True,NotMZ,MZ,51488_81352,51488,81352,...,3.6,101.12,86.45,0.0,38.6,71.69,71.76,1.76,0.0,1.76
2,2,100408,2,33,True,MZ,MZ,51730_81594,51730,81594,...,2,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68
3,3,100610,2,27,True,NotMZ,DZ,52813_82634,52813,82634,...,2,122.25,110.45,0.0,38.6,84.84,85.31,1.92,1.0,1.88
4,4,101006,1,35,True,MZ,,51283_52850_81149,51283,81149,...,6,122.25,111.41,0.0,38.6,123.80,123.31,1.80,0.0,1.80


In [ ]:
df.tail()

,Unnamed: 0,Subject,Gender,Age_in_Yrs,HasGT,ZygositySR,ZygosityGT,Family_ID,Mother_ID,Father_ID,...,Noise_Comp,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final
1081,1081,992774,2,35,True,NotTwin,,51345_81210,51345,81210,...,8.4,122.25,111.41,4.0,50.1,107.17,103.55,1.76,0.0,1.76
1082,1082,993675,1,29,True,NotTwin,,55800_85621,55800,85621,...,0.4,122.25,110.45,0.0,38.6,84.07,84.25,1.80,1.0,1.76
1083,1083,994273,2,30,True,NotTwin,,52364_82227,52364,82227,...,6.0,122.25,111.41,7.0,63.8,110.65,109.73,1.80,1.0,1.76
1084,1084,995174,2,25,True,MZ,MZ,55923_85743,55923,85743,...,3.6,88.61,64.58,3.0,50.1,117.16,117.40,1.80,0.0,1.80
1085,1085,996782,1,28,True,NotMZ,DZ,51541_81405,51541,81405,...,6.0,108.79,97.19,0.0,38.6,75.43,73.72,1.84,0.0,1.84


Extract data according to a certain criteria

In [ ]:
df[df['ZygositySR']=='MZ'] # ex: all of our monozygotic twins!)

,Unnamed: 0,Subject,Gender,Age_in_Yrs,HasGT,ZygositySR,ZygosityGT,Family_ID,Mother_ID,Father_ID,...,Noise_Comp,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final
2,2,100408,2,33,True,MZ,MZ,51730_81594,51730,81594,...,2,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68
4,4,101006,1,35,True,MZ,,51283_52850_81149,51283,81149,...,6,122.25,111.41,0.0,38.6,123.80,123.31,1.80,0.0,1.80
11,11,102311,1,26,True,MZ,MZ,51679_81543,51679,81543,...,4.4,122.25,110.45,1.0,52.2,98.31,99.06,1.80,0.0,1.80
15,15,102816,1,30,True,MZ,MZ,51418_81283,51418,81283,...,5.2,122.25,111.41,1.0,52.6,98.31,97.68,1.72,1.0,1.68
16,16,103010,2,25,True,MZ,MZ,55895_85715,55895,85715,...,5.2,101.12,86.45,7.0,38.6,103.37,102.15,1.84,0.0,1.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,1068,971160,2,27,True,MZ,MZ,52796_82617,52796,82617,...,6.0,96.87,77.12,0.0,38.6,84.84,85.31,1.94,1.0,1.90
1069,1069,972566,1,35,True,MZ,,56077_85897,56077,85897,...,2.8,101.12,87.11,2.0,50.1,73.86,71.78,1.92,0.0,1.92
1070,1070,973770,2,25,True,MZ,MZ,55910_85730,55910,85730,...,5.2,108.79,97.19,0.0,51.9,126.51,128.53,1.92,0.0,1.92
1078,1078,990366,2,22,True,MZ,,56064_85885,56064,85885,...,4.4,108.79,97.19,1.0,52.6,93.82,92.63,1.80,0.0,1.80


In [ ]:
df[df['Age_in_Yrs']>30] # ex: all of our subjects that are over 30

,Unnamed: 0,Subject,Gender,Age_in_Yrs,HasGT,ZygositySR,ZygosityGT,Family_ID,Mother_ID,Father_ID,...,Noise_Comp,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final
2,2,100408,2,33,True,MZ,MZ,51730_81594,51730,81594,...,2,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68
4,4,101006,1,35,True,MZ,,51283_52850_81149,51283,81149,...,6,122.25,111.41,0.0,38.6,123.80,123.31,1.80,0.0,1.80
8,8,101915,1,35,True,NotTwin,,51977_81841,51977,81841,...,4.4,96.87,77.61,0.0,38.6,112.11,111.70,1.84,1.0,1.80
18,18,103212,2,34,True,NotTwin,,56105_85925,56105,85925,...,8.4,101.12,87.11,1.0,49.7,72.63,70.67,1.80,1.0,1.76
21,21,103818,1,32,True,MZ,MZ,51343_81208,51343,81208,...,4.4,108.79,98.04,0.0,38.6,90.63,87.88,1.88,1.0,1.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,1071,978578,1,34,True,NotTwin,,51945_81809,51945,81809,...,3.6,122.25,111.41,0.0,38.6,80.52,78.21,1.72,1.0,1.68
1072,1072,979984,1,34,True,NotTwin,,52164_82027_99980,52164,99980,...,4.4,122.25,111.41,5.0,54.3,86.03,84.40,1.76,0.0,1.76
1077,1077,989987,2,33,True,NotTwin,,52040_81904,52040,81904,...,2.0,101.12,87.11,4.0,52.6,98.31,97.68,1.80,0.0,1.80
1080,1080,992673,1,33,True,NotTwin,,56143_85963,56143,85963,...,3.6,122.25,111.41,1.0,38.6,101.63,99.26,1.80,0.0,1.80


Making sure all the dataframes are okay:

In [ ]:
print(f"df type: {type(df)}")
print(f"df shape: {df.shape}")

df type: <class 'pandas.core.frame.DataFrame'>
df shape: (1086, 597)


In [ ]:
df

,Unnamed: 0,Subject,Gender,Age_in_Yrs,HasGT,ZygositySR,ZygosityGT,Family_ID,Mother_ID,Father_ID,...,Noise_Comp,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final
0,0,100206,2,27,True,NotTwin,,56037_85858,56037,85858,...,6,108.79,97.19,1.0,49.7,72.63,72.03,1.84,0.0,1.84
1,1,100307,1,27,True,NotMZ,MZ,51488_81352,51488,81352,...,3.6,101.12,86.45,0.0,38.6,71.69,71.76,1.76,0.0,1.76
2,2,100408,2,33,True,MZ,MZ,51730_81594,51730,81594,...,2,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68
3,3,100610,2,27,True,NotMZ,DZ,52813_82634,52813,82634,...,2,122.25,110.45,0.0,38.6,84.84,85.31,1.92,1.0,1.88
4,4,101006,1,35,True,MZ,,51283_52850_81149,51283,81149,...,6,122.25,111.41,0.0,38.6,123.80,123.31,1.80,0.0,1.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,1081,992774,2,35,True,NotTwin,,51345_81210,51345,81210,...,8.4,122.25,111.41,4.0,50.1,107.17,103.55,1.76,0.0,1.76
1082,1082,993675,1,29,True,NotTwin,,55800_85621,55800,85621,...,0.4,122.25,110.45,0.0,38.6,84.07,84.25,1.80,1.0,1.76
1083,1083,994273,2,30,True,NotTwin,,52364_82227,52364,82227,...,6.0,122.25,111.41,7.0,63.8,110.65,109.73,1.80,1.0,1.76
1084,1084,995174,2,25,True,MZ,MZ,55923_85743,55923,85743,...,3.6,88.61,64.58,3.0,50.1,117.16,117.40,1.80,0.0,1.80


## 2. Documentation

Take a careful look at your data's **[documentation](https://wiki.humanconnectome.org/display/PublicData/HCP-YA+Data+Dictionary-+Updated+for+the+1200+Subject+Release)** to understand all of the variables. Unfortunately, thorough documentation is not always available. Some data types are also very field-specific and require the help of experts, which is part of what makes machine learning so wonderfully interdisciplinary.

Given that we have 597 variables, let's narrow this down a bit to make the downstream interpretation of our model more tractable (dimensionality reduction itself is a whole separate topic that you will learn about later).

In [ ]:
df.columns

Index(['Unnamed: 0', 'Subject', 'Gender', 'Age_in_Yrs', 'HasGT', 'ZygositySR',
       'ZygosityGT', 'Family_ID', 'Mother_ID', 'Father_ID',
       ...
       'Noise_Comp', 'Odor_Unadj', 'Odor_AgeAdj', 'PainIntens_RawScore',
       'PainInterf_Tscore', 'Taste_Unadj', 'Taste_AgeAdj', 'Mars_Log_Score',
       'Mars_Errs', 'Mars_Final'],
      dtype='object', length=597)

In [ ]:
# including some basic variables such as subject ID, gender, age in years, zygosity (self reported and genotyped), bed time (HH:MM:SS)
cognition = ['Subject','Gender','Age_in_Yrs','ZygositySR','ZygosityGT','PSQI_BedTime'] 

# let's now include all of our cognitive variables (quantitative values associated with a battery of 
    # psychological tests that are supposed to convey information about a subject's cognitive abilities)
for col in df.columns:
    if (col.find("PicSeq")!=-1 or col.find("CardSort")!=-1 or col.find("Flanker")!=-1
       or col.find("PMAT")!=-1 or col.find("ReadEng")!=-1 or col.find("PicVocab")!=-1
       or col.find("ProcSpeed")!=-1 or col.find("DDisc")!=-1 or col.find("VSPLOT")!=-1
       or col.find("SCPT")!=-1 or col.find("IWRD")!=-1 or col.find("ListSort")!=-1):
        cognition.append(col)
df_cognition = df[cognition]

In [ ]:
df_cognition

,Subject,Gender,Age_in_Yrs,ZygositySR,ZygosityGT,PSQI_BedTime,SCPT_Compl,IWRD_Compl,PMAT_Compl,VSPLOT_Compl,...,SCPT_FP,SCPT_FN,SCPT_TPRT,SCPT_SEN,SCPT_SPEC,SCPT_LRNR,IWRD_TOT,IWRD_RTC,ListSort_Unadj,ListSort_AgeAdj
0,100206,2,27,NotTwin,,22:30:00,True,True,True,True,...,5.0,2.0,520.0,0.9667,0.9583,9.0,34.0,1475.00,112.89,103.57
1,100307,1,27,NotMZ,MZ,22:00:00,True,True,True,True,...,5.0,1.0,412.0,0.9833,0.9583,11.0,35.0,1442.00,108.06,98.44
2,100408,2,33,MZ,MZ,22:00:00,True,True,True,True,...,10.0,8.0,468.0,0.8667,0.9167,9.0,37.0,1450.75,108.06,102.63
3,100610,2,27,NotMZ,DZ,21:30:00,True,True,True,True,...,3.0,0.0,465.0,1.0000,0.9750,7.0,38.0,1240.00,117.39,108.26
4,101006,1,35,MZ,,23:00:00,True,True,True,True,...,3.0,1.0,440.0,0.9833,0.9750,15.0,37.0,1579.50,93.90,84.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,992774,2,35,NotTwin,,22:00:00,True,True,True,True,...,13.0,4.0,482.0,0.9333,0.8917,8.0,34.0,2012.50,96.99,88.33
1082,993675,1,29,NotTwin,,22:00:00,True,True,True,True,...,1.0,1.0,490.0,0.9833,0.9917,14.0,38.0,1760.00,121.89,113.85
1083,994273,2,30,NotTwin,,2:00:00,True,True,True,True,...,2.0,3.0,480.0,0.9500,0.9833,11.0,31.0,1812.50,108.06,102.63
1084,995174,2,25,MZ,MZ,23:00:00,True,True,True,True,...,19.0,5.0,390.0,0.9167,0.8417,9.0,38.0,1373.00,108.06,98.44


## 3. Removing variables (columns) that you don't need

In [ ]:
df_cognition.columns

Index(['Subject', 'Gender', 'Age_in_Yrs', 'ZygositySR', 'ZygosityGT',
       'PSQI_BedTime', 'SCPT_Compl', 'IWRD_Compl', 'PMAT_Compl',
       'VSPLOT_Compl', 'PicSeq_Unadj', 'PicSeq_AgeAdj', 'CardSort_Unadj',
       'CardSort_AgeAdj', 'Flanker_Unadj', 'Flanker_AgeAdj', 'PMAT24_A_CR',
       'PMAT24_A_SI', 'PMAT24_A_RTCR', 'ReadEng_Unadj', 'ReadEng_AgeAdj',
       'PicVocab_Unadj', 'PicVocab_AgeAdj', 'ProcSpeed_Unadj',
       'ProcSpeed_AgeAdj', 'DDisc_SV_1mo_200', 'DDisc_SV_6mo_200',
       'DDisc_SV_1yr_200', 'DDisc_SV_3yr_200', 'DDisc_SV_5yr_200',
       'DDisc_SV_10yr_200', 'DDisc_SV_1mo_40K', 'DDisc_SV_6mo_40K',
       'DDisc_SV_1yr_40K', 'DDisc_SV_3yr_40K', 'DDisc_SV_5yr_40K',
       'DDisc_SV_10yr_40K', 'DDisc_AUC_200', 'DDisc_AUC_40K', 'VSPLOT_TC',
       'VSPLOT_CRTE', 'VSPLOT_OFF', 'SCPT_TP', 'SCPT_TN', 'SCPT_FP', 'SCPT_FN',
       'SCPT_TPRT', 'SCPT_SEN', 'SCPT_SPEC', 'SCPT_LRNR', 'IWRD_TOT',
       'IWRD_RTC', 'ListSort_Unadj', 'ListSort_AgeAdj'],
      dtype='object')

Here, we have two columns that seem to repeat the same information: self-reported (ZygositySR) vs genotyped (ZygosityGT) zygosity. Do we really need both? This could depend on what you want to look at. For instance, you may want to see how accurately your subjects self-report their zygosity, meaning that you would need to keep both variables to compare them to each other. However, let's say that for the sake of your analysis, you only really need to keep one zygosity variable. Which one should you keep?

In [ ]:
df_cognition['ZygositySR'].value_counts()

NotTwin    528
NotMZ      285
MZ         269
             4
Name: ZygositySR, dtype: int64

In [ ]:
df_cognition['ZygosityGT'].value_counts()

      649
MZ    275
DZ    162
Name: ZygosityGT, dtype: int64

Conclusion: let's remove ZygosityGT since it seems to have a lot of NAs.\
_Note: MZ = monozygotic twins (share ~100% of the same genes), DZ = dizygotic twins (share ~50% of the same genes), NotTwin = non-twin siblings (share ~50% of the same genes)._

In [ ]:
df_cognition.drop('ZygosityGT', inplace=True, axis=1)

/Users/nadiablostein/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
df_cognition

,Subject,Gender,Age_in_Yrs,ZygositySR,PSQI_BedTime,SCPT_Compl,IWRD_Compl,PMAT_Compl,VSPLOT_Compl,PicSeq_Unadj,...,SCPT_FP,SCPT_FN,SCPT_TPRT,SCPT_SEN,SCPT_SPEC,SCPT_LRNR,IWRD_TOT,IWRD_RTC,ListSort_Unadj,ListSort_AgeAdj
0,100206,2,27,NotTwin,22:30:00,True,True,True,True,125.07,...,5.0,2.0,520.0,0.9667,0.9583,9.0,34.0,1475.00,112.89,103.57
1,100307,1,27,NotMZ,22:00:00,True,True,True,True,110.61,...,5.0,1.0,412.0,0.9833,0.9583,11.0,35.0,1442.00,108.06,98.44
2,100408,2,33,MZ,22:00:00,True,True,True,True,125.71,...,10.0,8.0,468.0,0.8667,0.9167,9.0,37.0,1450.75,108.06,102.63
3,100610,2,27,NotMZ,21:30:00,True,True,True,True,109.04,...,3.0,0.0,465.0,1.0000,0.9750,7.0,38.0,1240.00,117.39,108.26
4,101006,1,35,MZ,23:00:00,True,True,True,True,84.68,...,3.0,1.0,440.0,0.9833,0.9750,15.0,37.0,1579.50,93.90,84.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,992774,2,35,NotTwin,22:00:00,True,True,True,True,89.89,...,13.0,4.0,482.0,0.9333,0.8917,8.0,34.0,2012.50,96.99,88.33
1082,993675,1,29,NotTwin,22:00:00,True,True,True,True,108.89,...,1.0,1.0,490.0,0.9833,0.9917,14.0,38.0,1760.00,121.89,113.85
1083,994273,2,30,NotTwin,2:00:00,True,True,True,True,87.10,...,2.0,3.0,480.0,0.9500,0.9833,11.0,31.0,1812.50,108.06,102.63
1084,995174,2,25,MZ,23:00:00,True,True,True,True,108.93,...,19.0,5.0,390.0,0.9167,0.8417,9.0,38.0,1373.00,108.06,98.44


## 4. One-hot encoding or binarizing categorical variables

To be machine-readable, your variables need to be numerical. However, if you just convert categorical variables to numerical values (ex: MZ=1, NotTwin=2 and DZ=3), you give a "distance" to the relationship between variables. For instance, since 1 is closer to 2 than to 3, you are telling your machine that MZ twins are "closer" to NotTwin siblings than to DZ twins. We want our categories to be independent. That's where one hot encoding comes into play: "[A representation of categorical variables as binary vectors](https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/)"

**(1)** Suppose you want to look at 3 categories of ZygositySR: MZ, DZ and NotTwin. Let's one-hot encode ZygositySR!

In [ ]:
one_hot = pd.get_dummies(df_cognition['ZygositySR'])

In [ ]:
one_hot

,,MZ,NotMZ,NotTwin
0,0,0,0,1
1,0,0,1,0
2,0,1,0,0
3,0,0,1,0
4,0,1,0,0
...,...,...,...,...
1081,0,0,0,1
1082,0,0,0,1
1083,0,0,0,1
1084,0,1,0,0


In [ ]:
one_hot=one_hot.drop(one_hot.columns[0], axis=1)
one_hot

,MZ,NotMZ,NotTwin
0,0,0,1
1,0,1,0
2,1,0,0
3,0,1,0
4,1,0,0
...,...,...,...
1081,0,0,1
1082,0,0,1
1083,0,0,1
1084,1,0,0


In [ ]:
df_tmp = df_cognition.drop('ZygositySR',axis = 1)  # Drop column ZygositySR as it is now encoded

In [ ]:
df_tmp = df_tmp.join(one_hot) # Join the encoded df

In [ ]:
df_tmp

,Subject,Gender,Age_in_Yrs,PSQI_BedTime,SCPT_Compl,IWRD_Compl,PMAT_Compl,VSPLOT_Compl,PicSeq_Unadj,PicSeq_AgeAdj,...,SCPT_SEN,SCPT_SPEC,SCPT_LRNR,IWRD_TOT,IWRD_RTC,ListSort_Unadj,ListSort_AgeAdj,MZ,NotMZ,NotTwin
0,100206,2,27,22:30:00,True,True,True,True,125.07,118.78,...,0.9667,0.9583,9.0,34.0,1475.00,112.89,103.57,0,0,1
1,100307,1,27,22:00:00,True,True,True,True,110.61,103.45,...,0.9833,0.9583,11.0,35.0,1442.00,108.06,98.44,0,1,0
2,100408,2,33,22:00:00,True,True,True,True,125.71,125.19,...,0.8667,0.9167,9.0,37.0,1450.75,108.06,102.63,1,0,0
3,100610,2,27,21:30:00,True,True,True,True,109.04,101.69,...,1.0000,0.9750,7.0,38.0,1240.00,117.39,108.26,0,1,0
4,101006,1,35,23:00:00,True,True,True,True,84.68,70.00,...,0.9833,0.9750,15.0,37.0,1579.50,93.90,84.01,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,992774,2,35,22:00:00,True,True,True,True,89.89,77.27,...,0.9333,0.8917,8.0,34.0,2012.50,96.99,88.33,0,0,1
1082,993675,1,29,22:00:00,True,True,True,True,108.89,101.07,...,0.9833,0.9917,14.0,38.0,1760.00,121.89,113.85,0,0,1
1083,994273,2,30,2:00:00,True,True,True,True,87.10,73.89,...,0.9500,0.9833,11.0,31.0,1812.50,108.06,102.63,0,0,1
1084,995174,2,25,23:00:00,True,True,True,True,108.93,101.12,...,0.9167,0.8417,9.0,38.0,1373.00,108.06,98.44,1,0,0


**(2)** Suppose that now, you only want to look at 2 categories of ZygositySR: MZ and Non-MZ (NotTwin and DZ). Instead of one-hot encoding (creating 2 variables), you can just binarize ZygositySR.

In [ ]:
df_cognition = df_cognition.replace('MZ',1)
df_cognition = df_cognition.replace('DZ',0)
df_cognition = df_cognition.replace('NotTwin',0)
df_cognition = df_cognition.replace('NotMZ',0)

In [ ]:
df_cognition['ZygositySR'].value_counts() # Mission accomplished!

0    813
1    269
       4
Name: ZygositySR, dtype: int64

## 5. Making your data machine-readable

Replace strings with integers/floats.

In [ ]:
df_cognition = df_cognition.replace('FALSE',0)
df_cognition = df_cognition.replace('TRUE',1)
df_cognition = df_cognition.replace(False,0)
df_cognition = df_cognition.replace(True,1)
df_cognition = df_cognition.replace('0',0)
df_cognition = df_cognition.replace('12',0)
df_cognition = df_cognition.replace('20',0)
df_cognition = df_cognition.replace('#NAME?',0)
df_cognition = df_cognition.replace(' ',np.NaN)

Convert your bed time variable from HH:MM:SS to seconds!

In [ ]:
df_cognition['PSQI_BedTime']

0       22:30:00
1       22:00:00
2       22:00:00
3       21:30:00
4       23:00:00
          ...   
1081    22:00:00
1082    22:00:00
1083     2:00:00
1084    23:00:00
1085    23:30:00
Name: PSQI_BedTime, Length: 1086, dtype: object

In [ ]:
ftr = [3600,60,1]
for i in range(len(df_cognition['PSQI_BedTime'])):
    x = sum([a*b for a,b in zip(ftr, map(int,df_cognition['PSQI_BedTime'][i].split(':')))])
    df_cognition['PSQI_BedTime'][i] = x

<ipython-input-32-a47667968683>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cognition['PSQI_BedTime'][i] = x


In [ ]:
df_cognition['PSQI_BedTime']

0       81000
1       79200
2       79200
3       77400
4       82800
        ...  
1081    79200
1082    79200
1083     7200
1084    82800
1085    84600
Name: PSQI_BedTime, Length: 1086, dtype: object

In [ ]:
df_cognition

,Subject,Gender,Age_in_Yrs,ZygositySR,PSQI_BedTime,SCPT_Compl,IWRD_Compl,PMAT_Compl,VSPLOT_Compl,PicSeq_Unadj,...,SCPT_FP,SCPT_FN,SCPT_TPRT,SCPT_SEN,SCPT_SPEC,SCPT_LRNR,IWRD_TOT,IWRD_RTC,ListSort_Unadj,ListSort_AgeAdj
0,100206,2,27,0.0,81000,1,1,1,1,125.07,...,5.0,2.0,520.0,0.9667,0.9583,9.0,34.0,1475.00,112.89,103.57
1,100307,1,27,0.0,79200,1,1,1,1,110.61,...,5.0,1.0,412.0,0.9833,0.9583,11.0,35.0,1442.00,108.06,98.44
2,100408,2,33,1.0,79200,1,1,1,1,125.71,...,10.0,8.0,468.0,0.8667,0.9167,9.0,37.0,1450.75,108.06,102.63
3,100610,2,27,0.0,77400,1,1,1,1,109.04,...,3.0,0.0,465.0,1.0000,0.9750,7.0,38.0,1240.00,117.39,108.26
4,101006,1,35,1.0,82800,1,1,1,1,84.68,...,3.0,1.0,440.0,0.9833,0.9750,15.0,37.0,1579.50,93.90,84.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,992774,2,35,0.0,79200,1,1,1,1,89.89,...,13.0,4.0,482.0,0.9333,0.8917,8.0,34.0,2012.50,96.99,88.33
1082,993675,1,29,0.0,79200,1,1,1,1,108.89,...,1.0,1.0,490.0,0.9833,0.9917,14.0,38.0,1760.00,121.89,113.85
1083,994273,2,30,0.0,7200,1,1,1,1,87.10,...,2.0,3.0,480.0,0.9500,0.9833,11.0,31.0,1812.50,108.06,102.63
1084,995174,2,25,1.0,82800,1,1,1,1,108.93,...,19.0,5.0,390.0,0.9167,0.8417,9.0,38.0,1373.00,108.06,98.44


Making sure every column is of type float (the next lines of code will throw an error if you forgot to replace any strings, and it will tell you what those strings are).

In [ ]:
for col in df_cognition.columns:
    df_cognition[col] = df_cognition[col].astype(float)

## 6. Handling not available (NA) and inf data:

Sometimes, Python will convert some of your values to + or - infinity, which will result in downstream errors. Convert them to NA, and then handle them as NA values.

In [ ]:
df_cognition = df_cognition.replace([np.inf, -np.inf], np.nan)

Next, you need to deal with your NA values. Usually, people replace them with the median value of your variable of interest. However, if a variable has too many NAs, you may want to remove it completely.

In [ ]:
df_cognition.isna().sum()

Subject               0
Gender                0
Age_in_Yrs            0
ZygositySR            4
PSQI_BedTime          0
SCPT_Compl            0
IWRD_Compl            0
PMAT_Compl            0
VSPLOT_Compl          0
PicSeq_Unadj          1
PicSeq_AgeAdj         1
CardSort_Unadj        2
CardSort_AgeAdj       2
Flanker_Unadj         0
Flanker_AgeAdj        0
PMAT24_A_CR           9
PMAT24_A_SI           9
PMAT24_A_RTCR         9
ReadEng_Unadj         0
ReadEng_AgeAdj        0
PicVocab_Unadj        0
PicVocab_AgeAdj       0
ProcSpeed_Unadj       0
ProcSpeed_AgeAdj      0
DDisc_SV_1mo_200      8
DDisc_SV_6mo_200      8
DDisc_SV_1yr_200      8
DDisc_SV_3yr_200      8
DDisc_SV_5yr_200      8
DDisc_SV_10yr_200     8
DDisc_SV_1mo_40K      8
DDisc_SV_6mo_40K      8
DDisc_SV_1yr_40K      8
DDisc_SV_3yr_40K      8
DDisc_SV_5yr_40K      8
DDisc_SV_10yr_40K     8
DDisc_AUC_200         8
DDisc_AUC_40K         8
VSPLOT_TC             8
VSPLOT_CRTE           8
VSPLOT_OFF            8
SCPT_TP         

Define a threshold for the minimal number of missing values a variable needs to have to qualify for removal from the dataset. Here, 12 is quite stringent.

In [ ]:
threshold=12

In [ ]:
remove_cols = []
for i in range(len(df_cognition.columns)):
    if (df_cognition.iloc[:,i].isnull().sum() >= threshold):
        remove_cols.append(df_cognition.columns[i])
df_cognition = df_cognition.drop(columns=remove_cols)

In [ ]:
df_cognition.isna().sum()

Subject              0
Gender               0
Age_in_Yrs           0
ZygositySR           4
PSQI_BedTime         0
SCPT_Compl           0
IWRD_Compl           0
PMAT_Compl           0
VSPLOT_Compl         0
PicSeq_Unadj         1
PicSeq_AgeAdj        1
CardSort_Unadj       2
CardSort_AgeAdj      2
Flanker_Unadj        0
Flanker_AgeAdj       0
PMAT24_A_CR          9
PMAT24_A_SI          9
PMAT24_A_RTCR        9
ReadEng_Unadj        0
ReadEng_AgeAdj       0
PicVocab_Unadj       0
PicVocab_AgeAdj      0
ProcSpeed_Unadj      0
ProcSpeed_AgeAdj     0
DDisc_SV_1mo_200     8
DDisc_SV_6mo_200     8
DDisc_SV_1yr_200     8
DDisc_SV_3yr_200     8
DDisc_SV_5yr_200     8
DDisc_SV_10yr_200    8
DDisc_SV_1mo_40K     8
DDisc_SV_6mo_40K     8
DDisc_SV_1yr_40K     8
DDisc_SV_3yr_40K     8
DDisc_SV_5yr_40K     8
DDisc_SV_10yr_40K    8
DDisc_AUC_200        8
DDisc_AUC_40K        8
VSPLOT_TC            8
VSPLOT_CRTE          8
VSPLOT_OFF           8
SCPT_TP              8
SCPT_TN              8
SCPT_FP    

Finally, you can replace your NA values with the variable-specific median (you can also use the mean, but the median is more robust against outliers)

In [ ]:
for col in df_cognition.columns:
    df_cognition[col].fillna(df_cognition[col].median(), inplace=True)

In [ ]:
df_cognition.isna().sum()

Subject              0
Gender               0
Age_in_Yrs           0
ZygositySR           0
PSQI_BedTime         0
SCPT_Compl           0
IWRD_Compl           0
PMAT_Compl           0
VSPLOT_Compl         0
PicSeq_Unadj         0
PicSeq_AgeAdj        0
CardSort_Unadj       0
CardSort_AgeAdj      0
Flanker_Unadj        0
Flanker_AgeAdj       0
PMAT24_A_CR          0
PMAT24_A_SI          0
PMAT24_A_RTCR        0
ReadEng_Unadj        0
ReadEng_AgeAdj       0
PicVocab_Unadj       0
PicVocab_AgeAdj      0
ProcSpeed_Unadj      0
ProcSpeed_AgeAdj     0
DDisc_SV_1mo_200     0
DDisc_SV_6mo_200     0
DDisc_SV_1yr_200     0
DDisc_SV_3yr_200     0
DDisc_SV_5yr_200     0
DDisc_SV_10yr_200    0
DDisc_SV_1mo_40K     0
DDisc_SV_6mo_40K     0
DDisc_SV_1yr_40K     0
DDisc_SV_3yr_40K     0
DDisc_SV_5yr_40K     0
DDisc_SV_10yr_40K    0
DDisc_AUC_200        0
DDisc_AUC_40K        0
VSPLOT_TC            0
VSPLOT_CRTE          0
VSPLOT_OFF           0
SCPT_TP              0
SCPT_TN              0
SCPT_FP    

## 7. Removing columns with a standard deviation of 0:

In [ ]:
df_cognition.std()

Subject              272924.716578
Gender                    0.498353
Age_in_Yrs                3.696538
ZygositySR                0.431874
PSQI_BedTime          33895.899248
SCPT_Compl                0.104583
IWRD_Compl                0.085551
PMAT_Compl                0.090698
VSPLOT_Compl              0.085551
PicSeq_Unadj             13.259626
PicSeq_AgeAdj            16.517525
CardSort_Unadj           10.405410
CardSort_AgeAdj           9.959837
Flanker_Unadj            10.124983
Flanker_AgeAdj           10.082648
PMAT24_A_CR               4.845544
PMAT24_A_SI               3.972801
PMAT24_A_RTCR          9231.671384
ReadEng_Unadj            10.662897
ReadEng_AgeAdj           14.921269
PicVocab_Unadj            9.760459
PicVocab_AgeAdj          15.471830
ProcSpeed_Unadj          15.313322
ProcSpeed_AgeAdj         19.898274
DDisc_SV_1mo_200         47.405074
DDisc_SV_6mo_200         56.169515
DDisc_SV_1yr_200         62.678330
DDisc_SV_3yr_200         46.275664
DDisc_SV_5yr_200    

In [ ]:
df_cognition = df_cognition.loc[:, df_cognition.std() > 0]

## 8. Feature scaling

You usually need to perform some sort of feature scaling to make sure that all of your variables are in the same range (this affects gradient-descent-based algorithms and distance-based algorithms, see link above).\
**Min-Max Scaling / Normalization:** X' = (X-Xmin) / (Xmax-Xmin), X' always ends up with a range of \[0,1\] \
**Standardization / Standard Scaler / Z-score):** X' = (X-mu)/sigma \
Which to use? Depends on your data! ["Normalization is good to use when you know that the distribution of your data does not follow a Gaussian distribution. Standardization, on the other hand, can be helpful in cases where the data follows a Gaussian distribution."](https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/)\
[Other popular scaling techniques](https://www.analyticsvidhya.com/blog/2020/07/types-of-feature-transformation-and-scaling/) include the log transform (you often see this with GWAS - Genome-Wide Association Studies) and dividing your column-wise values by the absolute value of the maximal value of each column (max abs scaler).

Example 1: Min-Max Scaling

In [ ]:
minMaxScaled_df_cognition=(df_cognition-df_cognition.min())/(df_cognition.max()-df_cognition.min())

Example 2: Standard Scaling

In [ ]:
standardized_df_cognition=(df_cognition-df_cognition.mean())/df_cognition.std()

Example 3: Sklearn Min-Max Scaler\
Slightly different from the Min-Max Scaling defined above:\
`
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min) + min
`

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()
mms.fit(df_cognition)
df_cognition_mms=mms.transform(df_cognition)

# 9. Loading local files into Google Colab

Nice tutorial [here](https://neptune.ai/blog/google-colab-dealing-with-files)

**TLDR:**

In [ ]:
from google.colab import files # files module from google.colab library

In [ ]:
uploaded = files.upload # you can then select files and upload them to your colab